<a href="https://colab.research.google.com/github/jfmaramburo/BEDU_COVID19/blob/main/COVID19_leerCSV_guardarMYSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Declaración de Imports
import pandas as pd
#!pip install sqlalchemy
import sqlalchemy
from sqlalchemy import create_engine
#!pip install pymysql
import pymysql
#!pip install -U PyYAML
import yaml 
#!pip install mysql.connector
import mysql.connector

In [3]:
#Leer informacion desde archivo CSV
df = pd.read_csv("/content/drive/MyDrive/Proyecto BEDU/Pitch/210803COVID19MEXICO.csv")

In [4]:
#Revisamos total de Registros y Tipos de Datos
total_registros = len(df.index)
print('Total de Regitros Leidos:',total_registros)
df.dtypes

Total de Regitros Leidos: 8549081


FECHA_ACTUALIZACION      object
ID_REGISTRO              object
ORIGEN                    int64
SECTOR                    int64
ENTIDAD_UM                int64
SEXO                      int64
ENTIDAD_NAC               int64
ENTIDAD_RES               int64
MUNICIPIO_RES             int64
TIPO_PACIENTE             int64
FECHA_INGRESO            object
FECHA_SINTOMAS           object
FECHA_DEF                object
INTUBADO                  int64
NEUMONIA                  int64
EDAD                      int64
NACIONALIDAD              int64
EMBARAZO                  int64
HABLA_LENGUA_INDIG        int64
INDIGENA                  int64
DIABETES                  int64
EPOC                      int64
ASMA                      int64
INMUSUPR                  int64
HIPERTENSION              int64
OTRA_COM                  int64
CARDIOVASCULAR            int64
OBESIDAD                  int64
RENAL_CRONICA             int64
TABAQUISMO                int64
OTRO_CASO                 int64
TOMA_MUE

In [5]:
#Validamos que no se encuentre ningun registro NA
df.isna().sum()

FECHA_ACTUALIZACION      0
ID_REGISTRO              0
ORIGEN                   0
SECTOR                   0
ENTIDAD_UM               0
SEXO                     0
ENTIDAD_NAC              0
ENTIDAD_RES              0
MUNICIPIO_RES            0
TIPO_PACIENTE            0
FECHA_INGRESO            0
FECHA_SINTOMAS           0
FECHA_DEF                0
INTUBADO                 0
NEUMONIA                 0
EDAD                     0
NACIONALIDAD             0
EMBARAZO                 0
HABLA_LENGUA_INDIG       0
INDIGENA                 0
DIABETES                 0
EPOC                     0
ASMA                     0
INMUSUPR                 0
HIPERTENSION             0
OTRA_COM                 0
CARDIOVASCULAR           0
OBESIDAD                 0
RENAL_CRONICA            0
TABAQUISMO               0
OTRO_CASO                0
TOMA_MUESTRA_LAB         0
RESULTADO_LAB            0
TOMA_MUESTRA_ANTIGENO    0
RESULTADO_ANTIGENO       0
CLASIFICACION_FINAL      0
MIGRANTE                 0
P

In [6]:
#Diccionario de Columnas que no ocuparemos
col_borrar = ['ORIGEN'
             ,'ENTIDAD_UM'
             ,'ENTIDAD_NAC'
             ,'TIPO_PACIENTE'
             ,'FECHA_SINTOMAS'
             ,'FECHA_DEF'
             ,'INTUBADO'
             ,'NEUMONIA'
             ,'EDAD'
             ,'NACIONALIDAD'             
             ,'EMBARAZO'                 
             ,'HABLA_LENGUA_INDIG'       
             ,'INDIGENA'                 
             ,'DIABETES'                 
             ,'EPOC'                     
             ,'ASMA'                     
             ,'INMUSUPR'                 
             ,'HIPERTENSION'             
             ,'OTRA_COM'                 
             ,'CARDIOVASCULAR'           
             ,'OBESIDAD'                 
             ,'RENAL_CRONICA'            
             ,'TABAQUISMO'               
             ,'OTRO_CASO'                
             ,'TOMA_MUESTRA_LAB'         
             ,'TOMA_MUESTRA_ANTIGENO'    
             ,'RESULTADO_ANTIGENO'    
             ,'MIGRANTE'              
             ,'PAIS_ORIGEN'
              ,'UCI']

#Pasar a nuevo data frame informacion con eliminado de columnas
df_mexico = df.drop(columns=col_borrar)

In [7]:
#convertimos los tipos de datos a fecha y validamos tipos de Datos:
diccionario_de_conversion = {
    'FECHA_ACTUALIZACION' : 'datetime64[ns]',
    'FECHA_INGRESO' : 'datetime64[ns]'
}

df_mexico = df_mexico.astype(diccionario_de_conversion)
df_mexico.dtypes

FECHA_ACTUALIZACION    datetime64[ns]
ID_REGISTRO                    object
SECTOR                          int64
SEXO                            int64
ENTIDAD_RES                     int64
MUNICIPIO_RES                   int64
FECHA_INGRESO          datetime64[ns]
RESULTADO_LAB                   int64
CLASIFICACION_FINAL             int64
PAIS_NACIONALIDAD              object
dtype: object

In [8]:
#Realizamos Filtrado por Clasificacion y Nacionalidad = México:
# 1: CASO DE COVID-19 CONFIRMADO POR ASOCIACIÓN CLÍNICA EPIDEMIOLÓGICA
# 2: CASO DE COVID-19 CONFIRMADO POR COMITÉ DE  DICTAMINACIÓN
# 3: CASO DE SARS-COV-2  CONFIRMADO
# 6: CASO SOSPECHOSO

clasificaciones = [1,2,3,6]
df_mexico = df_mexico[df_mexico.CLASIFICACION_FINAL.isin (clasificaciones)& (df_mexico.PAIS_NACIONALIDAD == 'México') ]

total_mexico = len(df_mexico.index)
print('Total de Casos en México:',total_mexico)

Total de Casos en México: 3250626


In [9]:
#Obtener DataFrame por Fecha de Ingreso, Entidad Federativa y tipo de Sexo
df_res_mexico = df_mexico.groupby(['FECHA_INGRESO', 'ENTIDAD_RES','SEXO'] ).size().reset_index(name='CASOS')

In [10]:
df_res_mexico

,FECHA_INGRESO,ENTIDAD_RES,SEXO,CASOS
0,2020-01-01,9,2,1
1,2020-01-01,11,2,1
2,2020-01-01,15,1,1
3,2020-01-01,18,1,1
4,2020-01-01,19,1,2
...,...,...,...,...
35339,2021-08-03,29,2,2
35340,2021-08-03,30,1,2
35341,2021-08-03,30,2,1
35342,2021-08-03,31,2,1


In [11]:
#Obtener datos de conexion
with open("/content/drive/MyDrive/Proyecto BEDU/Pitch/config.yaml", "r") as f:
     params = yaml.load(f, Loader=yaml.FullLoader)
    
#Guadardar DataFrame DF_RES_MEXICO en Mysql
database_username = params['username']
database_password = params['password']
database_ip       = params['host']
database_port     = params['port']
database_name     = params['database']
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}:10001/{3}'.
                                               format(database_username, database_password, database_ip,  database_name))

df_res_mexico.to_sql(con=database_connection, name='casos_global', if_exists='replace')

In [12]:
df_sonora = df_mexico[(df_mexico.ENTIDAD_RES == 26) ]
df_res_mexico = df_sonora.groupby(['FECHA_INGRESO', 'MUNICIPIO_RES','SEXO'] ).size().reset_index(name='CASOS')
df_res_mexico

,FECHA_INGRESO,MUNICIPIO_RES,SEXO,CASOS
0,2020-01-01,30,1,1
1,2020-01-02,55,2,1
2,2020-01-05,30,2,1
3,2020-01-06,30,2,1
4,2020-01-06,42,2,1
...,...,...,...,...
12824,2021-08-02,70,1,5
12825,2021-08-02,70,2,2
12826,2021-08-02,71,2,1
12827,2021-08-03,29,1,1


In [13]:
#Obtener datos de conexion
with open("/content/drive/MyDrive/Proyecto BEDU/Pitch/config.yaml", "r") as f:
     params = yaml.load(f, Loader=yaml.FullLoader)
    
#Guadardar DataFrame DF_RES_SONORA en Mysql
database_username = params['username']
database_password = params['password']
database_ip       = params['host']
database_port     = params['port']
database_name     = params['database']
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}:10001/{3}'.
                                               format(database_username, database_password, database_ip,  database_name))

df_res_mexico.to_sql(con=database_connection, name='casos_sonora', if_exists='replace')